<a href="https://colab.research.google.com/github/Bk073/TensorFlow-Tutorials/blob/master/mnist_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The MNIST data has handwritten digits from 0–9 with 60,000 images for training
and 10,000 images for testing

The images are normalized to the size of 28 image pixels by 28
image pixels, converted to grey size, and centered to a fixed size

Load the MNIST data directly from TensorFlow. Note that we specify one hot
encoding as an argument while loading the data. The label is stored as integers
but should be loaded as one-hot encoding in-order to train.

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


A placeholder is a tensor where the **data is passed**. **Placeholders aren't specific
values but will receive input during computation .** 

x_input is the input where the images will be fed later.
y_input is the placeholder where the one-shot labels or targets will be supplied.

The None in the shape argument indicates that it can be of any size as we have not
yet defined the batch size.**bold text**

None = Dynamic shape

In [0]:
# 64 * 64 = 784 i.e dimension of image. Now the input layer must have 784 nodes

input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200

x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

**Placeholder**

Blog:  https://blog.metaflow.fr/shapes-and-dynamic-dimensions-in-tensorflow-7b1fe79be363

 Its value must be fed using the feed_dict optional argument to Session.run(), Tensor.eval(), or Operation.run().
 
 sess.run(y, feed_dict={x: rand_array})
 
 source: docs
 
Blog : **To access a dynamic shape value, you need to run your graph and feed any placeholder that your tensor my depended upon:**

The values of these variables will be learned during
computation

In [0]:
weights = tf.Variable(tf.random_normal([input_size, no_classes]))
bias = tf.Variable(tf.random_normal([no_classes]))

The inputs are then weighted and added with the bias to
produce logits as shown next:

In [0]:
logits = tf.matmul(x_input, weights) + bias

**Loss function**

The loss can be computed by averaging the cross-entropies. Then the cross-
entropy is fed through gradient descent optimization done
by tf.train.GradientDescentOptimizer . The optimizer takes the loss and minimizes it
with a learning rate of 0.5 . The computation of softmax, cross-entropy, loss,
optimization is shown next:

**tf.nn.softmax_cross_entropy_with_logits**

-> it is better to use softmax coupled
with cross-entropy for comparing logits and one-hot labels.

Measures the probability error in discrete classification tasks in which the classes are mutually exclusive (each entry is in exactly one class).

NOTE: While the classes are mutually exclusive, their probabilities need not be. All that is required is that each row of labels is a valid probability distribution. If they are not, the computation of the gradient will be incorrect.



In [9]:
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=logits)
loss_operation = tf.reduce_mean(softmax_cross_entropy)
optimiser = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss_operation)

W0711 15:13:19.197251 140649360131968 deprecation.py:323] From <ipython-input-9-e908baccd1c9>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



**Training model ** and **Session**

Initialize the variables using a global variable initializer

In [0]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [12]:
for batch_no in range(total_batches):
  mnist_batch = mnist_data.train.next_batch(batch_size)
  loss_value = session.run([optimiser, loss_operation], feed_dict={
      x_input: mnist_batch[0],
      y_input: mnist_batch[1]
  })
  print(loss_value)

[None, 13.149568]
[None, 12.686928]
[None, 11.001388]
[None, 8.908009]
[None, 9.513539]
[None, 8.781652]
[None, 7.209237]
[None, 5.9131727]
[None, 6.033996]
[None, 6.26871]
[None, 5.2980742]
[None, 4.4977183]
[None, 5.415235]
[None, 5.1726627]
[None, 5.2123895]
[None, 4.983762]
[None, 4.6331716]
[None, 5.0341663]
[None, 3.4482338]
[None, 4.361772]
[None, 3.5199456]
[None, 3.4030077]
[None, 3.4574268]
[None, 2.7100787]
[None, 2.1369193]
[None, 3.512426]
[None, 3.9308865]
[None, 2.240956]
[None, 2.138719]
[None, 2.4432073]
[None, 2.5776627]
[None, 2.6466565]
[None, 2.6281087]
[None, 2.2433355]
[None, 2.4506707]
[None, 2.4943836]
[None, 2.1070054]
[None, 1.8580148]
[None, 2.1389234]
[None, 2.1950686]
[None, 2.02647]
[None, 2.711564]
[None, 2.3337216]
[None, 2.4277363]
[None, 2.0814385]
[None, 2.0795078]
[None, 2.446528]
[None, 1.5046741]
[None, 2.065237]
[None, 2.1206264]
[None, 1.9071944]
[None, 1.4844735]
[None, 1.833798]
[None, 1.6980351]
[None, 1.6585736]
[None, 2.3372726]
[None, 0.91

In [13]:
predictions = tf.argmax(logits, 1)
correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
accuracy_value = session.run(accuracy_operation, feed_dict={
    x_input: test_images,
    y_input: test_labels
})
print('Accuracy:', accuracy_value)
session.close()

Accuracy: 0.8086
